<a href="https://colab.research.google.com/github/gopal2812/EPAiAssignment14/blob/main/session14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this project you have 4 files containing information about persons.

The files are:
* `personal_info.csv` -   personal information such as name, gender, etc. (one row per person)
* `vehicles.csv` -   what vehicle people own (one row per person)
* `employment.csv` -   where a person is employed (one row per person)
* `update_status.csv` -   when the person's data was created and last updated

Each file contains a key, `SSN`, which **uniquely** identifies a person.

This key is present in **all** four files.

You are guaranteed that the same SSN value is present in **every** file, and that it only appears **once per file**.

In addition, the files are all sorted by SSN, i.e. the SSN values appear in the same order in each file.


For this project you have 4 files containing information about persons.

The files are:

personal_info.csv - personal information such as name, gender, etc. (one row per person)
vehicles.csv - what vehicle people own (one row per person)
employment.csv - where a person is employed (one row per person)
update_status.csv - when the person's data was created and last updated
Each file contains a key, SSN, which uniquely identifies a person.

This key is present in all four files.

You are guaranteed that the same SSN value is present in every file, and that it only appears once per file.

In addition, the files are all sorted by SSN, i.e. the SSN values appear in the same order in each file.

##### Goal 1

Your first task is to create iterators for each of the four files that contained cleaned up data, of the correct type (e.g. string, int, date, etc), and represented by a named tuple.

For now these four iterators are just separate, independent iterators.

####Goal 2
Create a single iterable that combines all the columns from all the iterators.

The iterable should yield named tuples containing all the columns. Make sure that the SSN's across the files match!

All the files are guaranteed to be in SSN sort order, and every SSN is unique, and every SSN appears in every file.

Make sure the SSN is not repeated 4 times - one time per row is enough!

##### Goal 3

Next, you want to identify any stale records, where stale simply means the record has not been updated since 3/1/2017 (e.g. last update date < 3/1/2017). Create an iterator that only contains current records (i.e. not stale) based on the `last_updated` field from the `status_update` file.

##### Goal 4

Find the largest group of car makes for each gender.

Possibly more than one such group per gender exists (equal sizes).

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
from itertools import islice
from collections import namedtuple, Counter
from datetime import datetime

import csv

In [3]:
!ls /content/gdrive/MyDrive/employment.csv
basedir='/content/gdrive/MyDrive/'
empfile=basedir+'employment.csv'

/content/gdrive/MyDrive/employment.csv


#### Data Analysis

In [4]:
open(empfile, 'r').readlines()[:5]

['employer,department,employee_id,ssn\n',
 'Stiedemann-Bailey,Research and Development,29-0890771,100-53-9824\n',
 'Nicolas and Sons,Sales,41-6841359,101-71-4702\n',
 'Connelly Group,Research and Development,98-7952860,101-84-0356\n',
 'Upton LLC,Marketing,56-9817552,104-22-0928\n']

In [5]:
personalfile=basedir+'personal_info.csv'
open(personalfile, 'r').readlines()[:5]

['ssn,first_name,last_name,gender,language\n',
 '100-53-9824,Sebastiano,Tester,Male,Icelandic\n',
 '101-71-4702,Cayla,MacDonagh,Female,Lao\n',
 '101-84-0356,Nomi,Lipprose,Female,Yiddish\n',
 '104-22-0928,Justinian,Kunzelmann,Male,Dhivehi\n']

In [6]:
updatefile=basedir+'update_status.csv'
open(updatefile, 'r').readlines()[:5]

['ssn,last_updated,created\n',
 '100-53-9824,2017-10-07T00:14:42Z,2016-01-24T21:19:30Z\n',
 '101-71-4702,2017-01-23T11:23:17Z,2016-01-27T04:32:57Z\n',
 '101-84-0356,2017-10-04T11:21:30Z,2016-09-21T23:04:07Z\n',
 '104-22-0928,2017-03-28T12:38:29Z,2016-04-15T11:37:17Z\n']

In [7]:
vehiclefile=basedir+'vehicles.csv'
open(vehiclefile, 'r').readlines()[:5]

['ssn,vehicle_make,vehicle_model,model_year\n',
 '100-53-9824,Oldsmobile,Bravada,1993\n',
 '101-71-4702,Ford,Mustang,1997\n',
 '101-84-0356,GMC,Yukon,2005\n',
 '104-22-0928,Oldsmobile,Intrigue,2000\n']

In [8]:
# Defining the datatypes in the row
data_types_vehicles = ['STRING', 'STRING', 'STRING', 'DATE']
data_types_upstatus = ['STRING', 'DATETIME', 'DATETIME']
data_types_perinfo = ['STRING', 'STRING', 'STRING', 'STRING','STRING']
data_types_employee = ['STRING', 'STRING', 'STRING', 'STRING']

In [9]:
# Function to cast the value to the appropriate datatype
def cast(data_type, value):
    if data_type == 'DOUBLE':
        return float(value)
    elif data_type == 'INT':
        return int(value)
    elif data_type =='DATETIME':
        return datetime.strptime(value,'%Y-%m-%dT%H:%M:%S%z').date()
    elif data_type == 'DATE':
        return datetime.strptime(value,'%Y').date().year
    else:
        return str(value)

In [10]:
#Function which takes one row and sends the elements one by one for casting. Returns a list of casted elements 
def cast_row(data_types, data_row):
    return [cast(data_type, value) 
            for data_type, value in zip(data_types, data_row)]

In [11]:
# reading the file provided by filename and returning an iterator
def read_file(filename, data_types, header = False):
    with open(filename) as file:
        file_iter = iter(file)
        headers = next(file_iter).strip('\n').split(',')
        tuplename = filename.split('.')[0].split('/')[-1]
        named_tuple = namedtuple(tuplename, headers, defaults=(None,) * len(headers))
        if header:
            yield headers
        for line in file_iter:
            data = line.strip('\n').split(',')
            data = cast_row(data_types, data)
            iterdata = named_tuple(*data)
            yield iterdata

#### Solution 1
Build generators for each of the datasets

In [12]:
def get_employee():
  return read_file(empfile, data_types_employee)

In [13]:
# define generator to get data from function via next() operator
employee_gen = get_employee()

In [14]:
next(employee_gen) # generator yielding next data

employment(employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', ssn='100-53-9824')

In [15]:
def get_personalinfo():
  return read_file(personalfile, data_types_perinfo)

In [16]:
personalinfo_gen = get_personalinfo()

In [17]:
next(personalinfo_gen)

personal_info(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')

In [18]:
def get_updatefile():
  return read_file(updatefile, data_types_upstatus)


In [19]:
updateinfo_gen = get_updatefile()

In [20]:
next(updateinfo_gen)

update_status(ssn='100-53-9824', last_updated=datetime.date(2017, 10, 7), created=datetime.date(2016, 1, 24))

In [21]:
def get_vehiclefile():
  return read_file(vehiclefile, data_types_vehicles)

In [22]:
vehicleinfo_gen = get_vehiclefile()

In [23]:
next(vehicleinfo_gen)

vehicles(ssn='100-53-9824', vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)

#### Solution 2
combined_iter carries all the field variables present disjointly in individual dataset of 4 files supplied.Steps done are as follows:-
a) sort the each file by key as ssn
b) combine the individual named tuple in a list with a key value pair where key as ssn and remaining member as values.

In [24]:

## Function for sorting the iterator and returns a sorted iterator
def sorting(iterator, field):
    sorted_ = iter(sorted(iterator, key=lambda x: getattr(x, field), reverse = False))
    return (sorted_)

In [25]:
## Function for merging the individual namedtuple in a list
def merge(ntuples):
    m = {}
    for i in ntuples:
        m.update(i._asdict())
    M = namedtuple("combiner", m.keys())(*m.values())
    return M

In [26]:
## Combining the iterators and returning the value via a yield thus making the function a generator
def combiner(*args):
    for k in zip(*args):
        yield merge(list(k))

In [27]:
## Sorting the iterators w.r.t ssn
rows_perinfo_sort = sorting(get_personalinfo(), 'ssn')
rows_employ_sort = sorting(get_employee(),'ssn')
rows_updateStatus_sort = sorting(get_updatefile(),'ssn')
rows_vehicles_sort = sorting(get_vehiclefile(),'ssn')

In [28]:
# Printing the first 5 elements from the combined iterators
for index, rows in enumerate(islice(combiner(rows_perinfo_sort,rows_employ_sort, rows_updateStatus_sort, rows_vehicles_sort), 5)):
    print(index,'-->',rows)

0 --> combiner(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic', employer='"Crona', department=' Bogan and Beatty"', employee_id='Business Development', last_updated=datetime.date(2017, 10, 7), created=datetime.date(2016, 1, 24), vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)
1 --> combiner(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao', employer='"Fay', department=' Becker and Purdy"', employee_id='Training', last_updated=datetime.date(2017, 1, 23), created=datetime.date(2016, 1, 27), vehicle_make='Ford', vehicle_model='Mustang', model_year=1997)
2 --> combiner(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish', employer='"Ankunding', department=' Larson and Leannon"', employee_id='Legal', last_updated=datetime.date(2017, 10, 4), created=datetime.date(2016, 9, 21), vehicle_make='GMC', vehicle_model='Yukon', model_year=2005

Solution 3: Identify any stale records, where stale simply means the record has not been updated since 3/1/2017 (e.g. last update date < 3/1/2017). Create an iterator that only contains current records (i.e. not stale) based on the last_updated field from the status_update file

In [29]:
# A function to remove all the values below a particular date and return an iterator for the rest
def filiter(combi):
    date_ref = datetime.strptime('3/1/2017','%m/%d/%Y').date()
    for obj in combi:
        if obj.last_updated >= date_ref:
            yield obj
        else:
            continue

In [30]:
## Sorting the iterators
rows_perinfo_sort = sorting(get_personalinfo(), 'ssn')
rows_employ_sort = sorting(get_employee(),'ssn')
rows_updateStatus_sort = sorting(get_updatefile(),'ssn')
rows_vehicles_sort = sorting(get_vehiclefile(),'ssn')

In [31]:
## Combining the iterators
combined_iter = combiner(rows_perinfo_sort,rows_employ_sort, rows_updateStatus_sort, rows_vehicles_sort)

In [32]:
## An iterator for the filtered dates
fil = filiter(combined_iter)

In [34]:
## Printing the first 5 elements after filtering values less than a particular date
fil = filiter(combined_iter)
for index, rows in enumerate(islice(fil,5)):
    print(index,'-->',rows)

0 --> combiner(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic', employer='"Crona', department=' Bogan and Beatty"', employee_id='Business Development', last_updated=datetime.date(2017, 10, 7), created=datetime.date(2016, 1, 24), vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)
1 --> combiner(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish', employer='"Ankunding', department=' Larson and Leannon"', employee_id='Legal', last_updated=datetime.date(2017, 10, 4), created=datetime.date(2016, 9, 21), vehicle_make='GMC', vehicle_model='Yukon', model_year=2005)
2 --> combiner(ssn='104-22-0928', first_name='Justinian', last_name='Kunzelmann', gender='Male', language='Dhivehi', employer='"Marquardt', department=' Wehner and Heathcote"', employee_id='Legal', last_updated=datetime.date(2017, 3, 28), created=datetime.date(2016, 4, 15), vehicle_make='Oldsmobile', vehicle_model='Intri

#### Solution 4: Find the largest group of car makes for each gender.

In [35]:
def carmake_per_gender(it, gender):
  filt_data = [x.vehicle_make for x in it if (x.gender == gender)]
  _count = Counter(filt_data)
  return Counter(_count)
  #return {k: v for k, v in _count.items() if v == max(_count.values())}

In [36]:
## Sorting the iterators
rows_perinfo_sort = sorting(get_personalinfo(), 'ssn')
rows_employ_sort = sorting(get_employee(),'ssn')
rows_updateStatus_sort = sorting(get_updatefile(),'ssn')
rows_vehicles_sort = sorting(get_vehiclefile(),'ssn')

In [37]:
## Combining the iterators
combined_iter = combiner(rows_perinfo_sort,rows_employ_sort, rows_updateStatus_sort, rows_vehicles_sort)

In [38]:
cars_by_female = carmake_per_gender(combined_iter, 'Female')
# print 3 most common carmakes owned by female members

cars_by_female.most_common(3)

[('Ford', 48), ('Chevrolet', 48), ('Mitsubishi', 25)]

In [39]:
## Sorting the iterators
rows_perinfo_sort = sorting(get_personalinfo(), 'ssn')
rows_employ_sort = sorting(get_employee(),'ssn')
rows_updateStatus_sort = sorting(get_updatefile(),'ssn')
rows_vehicles_sort = sorting(get_vehiclefile(),'ssn')
## Combining the iterators
combined_iter = combiner(rows_perinfo_sort,rows_employ_sort, rows_updateStatus_sort, rows_vehicles_sort)
cars_by_male = carmake_per_gender(combined_iter, 'Male')
cars_by_male.most_common(3)

[('Ford', 44), ('Chevrolet', 38), ('GMC', 31)]

Ford is the largest carmake of the cars owned by male and female members.